In [1]:
!pip install -q -q -q geopandas
!pip install -q -q -q rtree

# Interpolation entre des données Insee et un fond de carte

Deux interpolations sont proposées. Dans la première, les statistiques territoriales sont obtenus en sommant les valeurs des carreaux intersectant les unités spatiales du fond de carte. Dans la deuxième, on rajoute une pondération, la part de la surface des carreaux intersectant les unités spatiales. On constate 


In [2]:
import geopandas as gpd
import shutil

In [5]:
shutil.unpack_archive('paris.zip', 'data')

Chargement des données carroyées et des quartiers de la politique de ville sur Paris

In [6]:
carreaux = gpd.read_file('data/paris.shp')

In [7]:
variables = ["Ind","Men","Men_pauv","Men_1ind","Men_5ind","Men_prop","Men_fmp","Ind_snv",
             "Men_surf","Men_coll","Men_mais","Log_av45","Log_45_70","Log_70_90","Log_ap90",
             "Log_inc","Log_soc","Ind_0_3","Ind_4_5","Ind_6_10","Ind_11_17","Ind_18_24",
             "Ind_25_39","Ind_40_54","Ind_55_64","Ind_65_79","Ind_80p","Ind_inc"]

In [8]:
carreaux = carreaux[["IdINSPIRE", "geometry"] + variables]

In [9]:
carreaux['surface'] = carreaux.geometry.area

In [10]:
qpv = gpd.read_file('data/qp_bdtopo_com_75056_2020.shp')

### Intersection spatiale entre les deux données spatiales (interpolation spatiale simple)

In [11]:
intersection = gpd.sjoin(qpv, carreaux, how='inner', op='intersects')

In [16]:
intersection.head(2)

,code,libelle,geometry,index_right,IdINSPIRE,Ind,Men,Men_pauv,Men_1ind,Men_5ind,...,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc,surface
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653802.699 6...",133,CRS3035RES200mN2885200E3761600,8.0,3.7,0.9,1.6,0.3,...,0.4,0.4,0.5,2.2,1.7,0.8,0.8,0.3,0.2,39989.301552
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653802.699 6...",559,CRS3035RES200mN2885200E3761800,520.0,194.0,34.0,59.0,28.0,...,39.5,56.0,39.0,95.0,97.0,62.0,53.0,36.0,8.0,39989.312182


Aggrégation des données par qpv

In [14]:
qpv_statistics = intersection.dissolve(by=['code', 'libelle'], aggfunc='sum', as_index=False)

In [17]:
qpv_statistics.head(2)

,code,libelle,geometry,index_right,Ind,Men,Men_pauv,Men_1ind,Men_5ind,Men_prop,...,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc,surface
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653802.699 6...",34133,5112.0,2150.7,444.9,889.6,237.3,141.8,...,365.4,498.9,294.0,1175.2,1126.7,494.8,394.8,173.3,96.2,319914.948929
1,QP075002,Chaufourniers,"POLYGON ((653876.162 6864327.010, 653913.377 6...",6625,10886.5,5048.0,955.0,2335.0,385.0,1482.0,...,570.0,822.0,545.5,2693.0,2265.0,1291.0,1276.0,445.0,257.5,239955.533755


### Intersection spatiale entre les deux données spatiales (au prorata de la surface)

In [18]:
intersection = gpd.overlay(qpv, carreaux)

In [19]:
ponderation = intersection.area / intersection.surface

In [20]:
intersection.loc[:,variables] = intersection.loc[:,variables].mul(ponderation, axis=0)

Aggrégation des données par qpv

In [21]:
qpv_statistics = intersection[['code', 'libelle', 'geometry'] + variables].dissolve(by=['code', 'libelle'], aggfunc='sum', as_index=False)

In [22]:
qpv_statistics.head(2)

,code,libelle,geometry,Ind,Men,Men_pauv,Men_1ind,Men_5ind,Men_prop,Men_fmp,...,Ind_4_5,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653882.944 6...",2264.187066,947.974481,183.343515,386.503798,103.230715,10.153103,141.051726,...,77.023592,156.675216,221.893923,124.053034,489.861762,504.837418,232.063625,193.580205,84.371875,43.396112
1,QP075002,Chaufourniers,"POLYGON ((654071.916 6864111.407, 654060.266 6...",2059.902066,939.976813,188.818222,414.423931,68.463556,252.474650,106.978945,...,43.697437,109.610218,156.805499,114.176747,481.204805,431.894507,239.231732,250.471354,89.689896,48.862052
